In [1]:
import sys
import random
from collections import defaultdict, namedtuple
from pathlib import Path
from textwrap import indent, dedent

INTERACTIVE = '/data/home/coclemen/projects/method-docstring-webapp'
if INTERACTIVE not in sys.path:
    sys.path.append(INTERACTIVE)

In [2]:
from server.model_init import FairseqModelInterface, Args

In [4]:
ROOT = Path("/smartml-athena/processed-data/fundef-docstrings/split/multi-task-translation")
MODEL_PATH = str(ROOT / "fairseq/2020-04-17/")
DATA_PATH = str(ROOT / "fairseq/binary")
TOKENIZER_ROOT = Path("/smartml-athena/processed-data/fundef-docstrings/dawn-py-pretrained/forColin")
TOKENIZER_PREFIX = str(TOKENIZER_ROOT / "hacky_byte_level")

args = Args(
        model_name_or_path=MODEL_PATH,
        data_name_or_path=DATA_PATH,
        tokenizer_prefix=TOKENIZER_PREFIX,
        checkpoint_file='checkpoint_best.pt',
)

model = FairseqModelInterface()
model.load_model(args)

In [183]:
REPLACE = {'Ġ': ' ', 'Ċ': '\n', 'ĉ': '\t', 'NEWLINE': '\n', '<user_handle>': 'param'}

def fix_whitespace(string):
    """Return tokenized data to string literal"""
    for r, f in REPLACE.items():
        string = string.replace(r, f)
    return string

def decode(outlines):
    """Reads gathered fairseq-generate lines, returns src, hyp, tgt"""
    return [
            fix_whitespace(''.join(outlines[0][1:])),
            fix_whitespace(''.join(outlines[1][1:])),
            fix_whitespace(''.join(outlines[2][2:]))
    ]

def put_together():
    assert False
    splitsrc = src.split('):\n')
    sig, body = splitsrc[0] + '):\n', '):\n'.join(splitsrc[1:])
    
    target = sig + '    """\n{}\n    """\n'.format(indent(tgt, ' '* 4)) + body
    hypothesis = sig + '    """\n{}\n    """\n'.format(indent(hyp, ' '* 4)) + body
    return [src, hypothesis, target]

def normalize_fairseq_schema(filepath):
    """Reads fairseq generated file and outputs results in the form of the GPT2 schema above"""
    gathered = defaultdict(list)
    with Path(filepath).open() as fin:
        for line in fin.readlines():
            if line[0] in ('S', 'T', 'H', 'P'):
                tokens = line.split()
                gathered[int(tokens[0].split('-')[1])].append(tokens)
    return [decode(gathered[k]) for k in gathered]

In [16]:
%run -i "~/projects/dawns_code/docstring_multitask_generation/example_functions_for_demos"

In [59]:
n_params = 0
for p in model.model.parameters():
    n_params += p.numel()
print(n_params / 1e6)

374.839488


In [62]:
p = list(model.model.parameters())

In [71]:
model.model.

GeneratorHubInterface(
  (models): ModuleList(
    (0): TransformerModel(
      (encoder): TransformerEncoder(
        (embed_tokens): Embedding(50181, 1472, padding_idx=1)
        (embed_positions): SinusoidalPositionalEmbedding()
        (layers): ModuleList(
          (0): TransformerEncoderLayer(
            (self_attn): MultiheadAttention(
              (k_proj): Linear(in_features=1472, out_features=1472, bias=True)
              (v_proj): Linear(in_features=1472, out_features=1472, bias=True)
              (q_proj): Linear(in_features=1472, out_features=1472, bias=True)
              (out_proj): Linear(in_features=1472, out_features=1472, bias=True)
            )
            (self_attn_layer_norm): _FusedLayerNorm(torch.Size([1472]), eps=1e-05, elementwise_affine=True)
            (fc1): Linear(in_features=1472, out_features=4096, bias=True)
            (fc2): Linear(in_features=4096, out_features=1472, bias=True)
            (final_layer_norm): _FusedLayerNorm(torch.Size([1472]

In [65]:
f = p[0]

In [70]:
f?

In [6]:
DOCSTRING_STYLES = {'oneline', 'numpydoc', 'google', 'restructuredtext', 'javadoc', 'oneparagraph', 'other'}

def code_format_docstring(docstring):
    if not docstring:
        return docstring
    if len(docstring.splitlines()) > 1 or docstring[0] == '\n':
        return indent(f'"""\n{docstring}\n"""', ' ' * 4)
    return indent(f'"""{docstring}"""', ' ' * 4)

def generate_method(example, model=model, beam=5, temperature=1.05):
    parts = ['# target body', example[0]]
    if example[1]:
        parts.append(code_format_docstring(example[1]))
    sig_doc = '\n'.join(parts)
    return model.translate(sig_doc, beam=beam, temperature=temperature)

def generate_docstring(example, style='numpydoc', model=model):
    assert style in DOCSTRING_STYLES
    method = '\n'.join([f'# target docstring style {style}', example[0], indent(example[2], ' ' * 4)])
    return model.translate(method, beam=7, temperature=1.)

In [138]:
OPENAI_CODEGEN = {}
OPENAI_CODEGEN['locate-3s'] = [
        'def return_indices_of_list_of_elements_that_are_3(l):',
        'Given the list l, return the indices of those list\n elements that have the value 3.',
        ''
]


OPENAI_CODEGEN['print-first-3'] = [
        'def print_first_3_characters_of_string(s):',
        '',
        ''
]

OPENAI_CODEGEN['first-3-to-int'] = [
        'def convert_first_3_characters_of_string_s_to_number(s):',
        '',
        ''
]

OPENAI_CODEGEN['reverse'] = [
        'def reverse_order_of_words(s):',
        '',
        ''
]

OPENAI_CODEGEN['reverse-for-loop'] = [
        'def reverse_list_using_explicit_for_loop(l):',
        '',
        ''
    
]


OPENAI_CODEGEN['return-odd'] = [
        'def return_odd_numbers_from_list(l):',
        'Given a list of integers, return only\n those which are odd.',
        ''
]


OPENAI_CODEGEN['return-odd-index-sum'] = [
        'def return_sum_of_odd_numbered_elements_of_list(l):',
        'Given the list l, return the sum of the elements\nthat occur at an odd-numbered index.',
        ''
]

OPENAI_CODEGEN['format-grades'] = [
        'def format_grades(f):',
        (
            'The file f contains the midterm grades for all the student in the class.\n'
            'Each line in the file has the format:\n'
            '    <Last Name>, "\t", <First Name>, "\t", <Score>\n'
            'This function reads the file and prints out each line in the format:\n'
            '    <Last Name>, "\t", <First Name>, "\t", <Score>\n'
        ),
        ''
]

OPENAI_CODEGEN['ask-user'] = [
        'def main():',
        'Asks the user for a number n and then\nprints the sum of all the numbers from 1 to n.',
        ''    
]

OPENAI_CODEGEN['alice-or-bob'] = [
        'def main():',
        'Asks the user for their name and then greets\nthem with their name, if their name is Alice or Bob.',
        ''
]

In [49]:
generate_method(DOCSTRING_EXAMPLES['load_file'])

"    try:\n        with codecs.open(file_path, 'r', 'utf-8') as f:\n            return f.read()\n    except Exception as e:\n        logger.warning(('Failed to load file %s: %s' % (file_path, e)))"

In [46]:
print(example[0] + '\n' + indent(example[2], ' '  * 4))

def add_ctrl(fold,tokenizer,sig='',docstring='',body=''):
    ctrl = get_ctrl(fold)

    sig = ' '.join(tokenizer.encode(sig).tokens)
    docstring = '+.join(tokenizer.encode(docstring).tokens)+'
    body = ' '.join(tokenizer.encode(body).tokens)
    third_to_str = {'sig':sig,'docstring':docstring,'body':body}

    inputs,outputs = get_i_o(fold)

    src = ctrl
    if len(inputs) == 1:
        src += ' '+third_to_str[list(inputs.keys())[0]]
    else:
        for third in inputs:
            src += ' '+third_to_str[third]
    return src


In [23]:
examples = [
    'def merge(left, right):',  # signature
    'Merge two sorted lists into one sorted list',  # docstring
    ''
]
method = generate_method(examples, beam=5)
print(generate_method(examples, beam=5))

    if (not left):
        return right
    if (not right):
        return left
    if (left[0] < right[0]):
        (left[0], right[0]) = (right[0], left[0])
    merge(left[1:], right[1:])
    return left


In [8]:
examples = [
    'def get_sum_of_odd_indices(l):',  # signature
    'return the sum of elements of list l which occur at odd indices'  # docstring
]
%time print(generate_method(examples))

    sum_of_odd_indices = 0
    for i in l:
        if (i % 2):
            sum_of_odd_indices += 1
    return sum_of_odd_indices
CPU times: user 777 ms, sys: 38.3 ms, total: 815 ms
Wall time: 814 ms


In [47]:
even_example = [
    'def any_large_elements(numbers):',
    '',
    '''
for number in numbers:
    if number > 1000:
        return True
return False'''
]
print(generate_docstring(even_example, style='numpydoc'))

    """
    Returns True if any of the numbers is larger than 1000.

    Parameters
    ----------
    numbers : list
        List of numbers.

    Returns
    -------
    bool
        True if any of the numbers is larger than 1000.
    """


In [31]:
url_examples = [
    'def find_all_urls(string, urlregex):',  # signature
    '', # docstring
]
url_examples.append(generate_method(url_examples))
print('METHOD PROMPT:\n\n')
print(url_examples[0])
print(url_examples[2])

print('\n')
for style in DOCSTRING_STYLES:
    print(f'GENERATED DOCSTRING STYLE = {style}')
    print(generate_docstring(url_examples, style))
    print('\n')

METHOD PROMPT:


def find_all_urls(string, urlregex):
    urls = []
    for match in re.finditer(urlregex, string):
        urls.append(match.group())
    return urls


GENERATED DOCSTRING STYLE = restructuredtext
    """

    :param string: 
    :param urlregex: 
    :return: 
    """


GENERATED DOCSTRING STYLE = google
    """
    Find all URLs in a string.

    Args:
        string (str): String to search for URLs.
        urlregex (str): Regex to search for URLs.

    Returns:
        list: List of URLs.
    """


GENERATED DOCSTRING STYLE = numpydoc
    """
    Find all URLs in a string.

    Parameters
    ----------
    string : str
        The string to search.
    urlregex : str
        The regular expression to search for URLs.

    Returns
    -------
    urls : list
        A list of all URLs in the string.
    """


GENERATED DOCSTRING STYLE = javadoc
    """
    @param string:
    @param urlregex:
    @return:
    """


GENERATED DOCSTRING STYLE = oneparagraph
    """
  

In [5]:
example = [
    'def sum_odd_elements(lst):',  # signature
    'sum elements of list which are odd',  # docstring
    '''sum = 0
for i in range(len(lst)):
    if (lst[i] % 2):
        sum += lst[i]
return sum
'''
]
print(generate_docstring(example, style='javadoc'))

NameError: name 'generate_docstring' is not defined

In [207]:
%run -i "aicodegen_examples.py"

In [200]:
sig

('    if (x == 0',)

In [208]:
examples = OPENAI_CODEGEN_EXAMPLES
print(examples.keys())
for name, example in examples.items():
    sig, docstring, _ = example
    method = generate_method(example, beam=5, temperature=1.0)
    print('\n'.join([sig, code_format_docstring(docstring), method]))
    print('\n')

dict_keys(['factorial', 'sum_to_n', 'palindrome', 'halfway', 'isprime', 'count_even_numbers', 'return_odd', 'return_indicies_of_3', 'any_large', 'non_zero_elements_indicies', 'same_chars'])
def factorial(x):
    """Compute the factorial of x"""
    if (x == 0):
        return 1
    else:
        return (factorial((x - 1)) * factorial((x - 2)))


def sum_to_n(n):
    """Sums the numbers from 1 to n"""
    sum = 0
    for i in range(1, (n + 1)):
        sum += i
    return sum


def is_palindrome(text):
    """Checks if given string is a palindrome"""
    if (len(text) != len(set(text))):
        return False
    for i in range(len(text)):
        if (text[i] != set(text[:i])):
            return False
    return True


def halfway(a, b):
    """What number is halfway between a and b?"""
    return ((a + b) / 2)


def is_prime(n):
    """Return true is a given number is prime, and false otherwise"""
    if (n < 2):
        return False
    if (n == 2):
        return True
    if ((n % 2)

In [12]:
FAIRSEQ_MODELDIR = Path('/smartml-athena/processed-data/fundef-docstrings/split/translate-method-docstring/fairseq')
TRANSLATION = FAIRSEQ_MODELDIR / '2020-02-25/generated/generate-test.txt'
dat_translation = normalize_fairseq_schema(TRANSLATION)

In [19]:
method_list = [d[0] for d in dat_translation]

In [259]:
def generate_docstring(method_text, beam=6, temp=1., model=model, **kwargs):
    print(model.translate(method_text, beam=beam, temperature=temp, **kwargs))

In [127]:
seen = []

In [240]:
key = random.randint(0, len(dat_translation)); seen.append(key)

print(f'key = {key}')
print('\n')
print(dat_translation[key][0])
print('\nhyp:\n')
print(dat_translation[key][1])
print('\ntgt:\n')
print(dat_translation[key][2])

key = 193352


def plot_grid(self, **kwargs):
    if ('ax' in kwargs):
        ax = kwargs.pop('ax')
    else:
        ax = self.ax
    if ('color' not in kwargs):
        kwargs['color'] = '0.5'
    lc = self.get_grid_line_collection(**kwargs)
    ax.add_collection(lc)
    return lc

hyp:

Plot the grid lines.

Parameters
----------
    kwargs : ax, colors.  The remaining kwargs are passed into the
        the LineCollection constructor.

Returns
-------
    lc : matplotlib.collections.LineCollection

tgt:

Add a grid to the current plot.

Parameters
----------
ax : matplotlib.axes.Axes, optional
    The axes to add the grid to.
color : string, optional
    The color of the grid lines.
ax : matplotlib.axes.Axes, optional
    The axes to add the grid to.

Returns
-------
ax : matplotlib.axes.Axes
    The axes added to the current plot.


# Docstring generation demo

In [241]:
example4 = method_list[193352]

In [187]:
key = 235413

In [ ]:
def parse_tweet(tweet_text, utc_offset):
    full_text = tweet_text[((tweet_text.index(('@%s' % USERNAME)) + len(('@%s' % USERNAME))) + 1):]
    reminder_message = parse_message(full_text)
    reminder_time = parse_time(full_text, utc_offset)
    logging.info(('parse_tweet: %s--%s' % (reminder_time.strip(), reminder_message.strip())))
    return (reminder_time.strip(), reminder_message.strip())

In [188]:
generate_docstring(method_list[key])

Parse a Tweet text and return a reminder_time and reminder_message.


In [189]:
key = 280783

In [ ]:
def loader_shift(loader, frame, relative=True):
    comp = loader.Comp()
    time = comp.TIME_UNDEFINED
    old_in = loader['GlobalIn'][time]
    old_out = loader['GlobalOut'][time]
    if relative:
        shift = frame
    else:
        shift = (frame - old_in)
    with preserve_inputs(loader, inputs=['ClipTimeStart', 'ClipTimeEnd', 'HoldFirstFrame', 'HoldLastFrame']):
        if (shift > 0):
            loader['GlobalOut'][time] = (old_out + shift)
            loader['GlobalIn'][time] = (old_in + shift)
        else:
            loader['GlobalIn'][time] = (old_in + shift)
            loader['GlobalOut'][time] = (old_out + shift)
    return int(shift)

In [190]:
generate_docstring(method_list[key], temp=1.0)

Given a frame of data, load the data to a new frame.

:param loader: The data loader to use.
:param frame: The frame to load.
:param relative: Whether to use relative time or absolute time (default True).
:return: The new frame number.


In [191]:
key = 294868

In [ ]:
def memoize(*args, **kwargs):
    if args:
        assert (len(args) == 1)
        assert (not kwargs)
        return memoize()(args[0])
    key_func = kwargs.pop('key_func', None)
    if kwargs:
        raise TypeError('memoize() got unexpected keyword arguments: %s', ', '.join(kwargs))

    def decorator(func):
        memory = {
        
        }

        @wraps(func)
        def wrapper(*args, **kwargs):
            key = (key_func(*args, **kwargs) if key_func else ((args + tuple(sorted(kwargs.items()))) if kwargs else args))
            try:
                return memory[key]
            except KeyError:
                try:
                    value = memory[key] = func(*args, **kwargs)
                    return value
                except SkipMemoization as e:
                    return (e.args[0] if e.args else None)
        wrapper.memory = memory
        return wrapper
    return decorator

In [192]:
generate_docstring(method_list[key])

Decorator to memoize function.

:param key_func: The function to use as a key for the memoization cache.
:type key_func: callable
:param args: Arguments to the function's __init__ method.
:type args: tuple
:param kwargs: Keyword arguments to the function's __init__ method.


In [193]:
key = 182441

In [ ]:
def get_access_token(self, http=None):
    if ((not self.access_token) or self.access_token_expired):
        if (not http):
            http = httplib2.Http()
        self.refresh(http)
    return AccessTokenInfo(access_token=self.access_token, expires_in=self._expires_in())

In [194]:
generate_docstring(method_list[key])

Return the access token and its expiration information.

If the token does not exist, get one.
If the token expired, refresh it.


In [ ]:
key = 334463

In [234]:
key = 269794
example3 = method_list[key]

In [ ]:
def parse_manifest(files, dirs):
    f = open(MANIFEST_IN, 'r')
    for line in f:
        line = line.strip()
        if ((len(line) <= 0) or (line[0] == '#')):
            continue
        tokens = line.split()
        if (len(tokens) >= 0):
            if (tokens[0] == 'include'):
                files.extend(tokens[1:])
            elif ((tokens[0] == 'recursive-include') and (tokens[2] == '*')):
                dirs.append(tokens[1])
            else:
                err = (((('Unsupported type ' + tokens[0]) + ' in ') + MANIFEST_IN) + ' file.')
                raise Exception(err)
    f.close()

In [216]:
generate_docstring(method_list[key])

Parse the contents of a MANIFEST_IN, which should be a list of files
to add to the MANIFEST_IN.


In [51]:
generate_docstring(
"""
def num_parameters(self, only_trainable: bool = False) -> int:
    if only_trainable:
        return int(sum(np.prod(w.shape.as_list()) for w in self.trainable_variables))
    else:
        return self.count_params()
""", beam=20, temp=1.
)

Return the number of parameters in this layer.

Parameters
----------
only_trainable : bool
    If True, return the number of trainable parameters.

Returns
-------
int
    Number of parameters in this layer.


In [54]:
generate_docstring(
"""
def get_activation(activation_string):
    if activation_string in ACT2FN:
        return ACT2FN[activation_string]
    else:
        raise KeyError("function {} not found in ACT2FN mapping {}".format(activation_string, list(ACT2FN.keys())))
""", beam=20, temp=1.5
)

Returns the string corresponding to the activation function defined by activation_string

Parameters
----------
activation_string : string
    String containing the name of the activation function

Returns
-------
activation_string : string
    String containing the string corresponding to the activation function defined by activation_string


# Dawn's examples

In [ ]:
def file_extension_counts(repo_dir):
    extension_counts = defaultdict(int)
    """return a dic of counts for all file extensions located under repo_dir"""
    for r, d, f in os.walk(repo_dir):
        for file in f:
            extension_counts[grab_extension(file)] += 1
    return extension_counts

In [179]:
generate_docstring(
"""
def file_extension_counts(repo_dir):
    extension_counts = defaultdict(int)

    for r, d, f in os.walk(repo_dir):
        for file in f:
            extension_counts[grab_extension(file)] += 1
    return extension_counts
"""
)

Returns a dictionary of file extension counts.

@param   repo_dir: string
@param  repo_dir: Path to repository.

@param:   dict
@param:  Dictionary of file extension counts.


In [ ]:
def grab_source_files_from_repo(repo,extension,quiet=True):
    """ Grabs all .extension filepaths under repo
    Args:
        repo: Path
            the head directory under which the .extension files are located
        extension: str or list of str extensions
            e.g. py or java
        quiet: bool
            whether to suppress prints of files that couldn't be loaded
    Returns:
        filepaths
    """
    all_raw_input_files = []
    try:
        if isinstance(extension, str):
            raw_input_files = Path(repo).glob('**/*.'+extension)
        else:
            extension = ['.'+ext for ext in extension]
            raw_input_files = Path(repo).glob('**/*')
            raw_input_files = [p for p in raw_input_files if grab_extension(p) in extension]
        res = [str(p) for p in raw_input_files if p.is_file()]
        return res
    except OSError:
        if not quiet:
            print("couldn't process "+str(repo))
        return []

In [232]:
example2 = (
"""    
def grab_source_files_from_repo(repo,extension,quiet=True):
    all_raw_input_files = []
    try:
        if isinstance(extension, str):
            raw_input_files = Path(repo).glob('**/*.'+extension)
        else:
            extension = ['.'+ext for ext in extension]
            raw_input_files = Path(repo).glob('**/*')
            raw_input_files = [p for p in raw_input_files if grab_extension(p) in extension]
        res = [str(p) for p in raw_input_files if p.is_file()]
        return res
    except OSError:
        if not quiet:
            print("couldn't process "+str(repo))
        return []
"""
)

In [ ]:
def read_n_lines_at_a_time(f,n):
    """read f in chunks to use pool while streaming giant files
    inputs:
        f (filehandle for open file) -- file you want to read
        n (int) -- number of lines to read at a time
       
    returns:
        generator with f broken into chunks
    """
    while True:
        lines = [f.readline() for i in range(n)]
        lines = [line for line in lines if line != '']
        if len(lines) > 0:
            yield [line for line in lines if line != '']
        else:
            break

In [228]:
example1 = """
def read_n_lines_at_a_time(f,n):
    while True:
        lines = [f.readline() for i in range(n)]
        lines = [line for line in lines if line != '']
        if len(lines) > 0:
            yield [line for line in lines if line != '']
        else:
            break
"""

# Examples Follow
&nbsp;  
&nbsp;  
&nbsp;  
&nbsp;  
&nbsp;  
&nbsp;  
&nbsp;  
&nbsp;  
&nbsp;  
&nbsp;  
&nbsp;  
&nbsp;  
&nbsp;  
&nbsp;  
&nbsp;  
&nbsp;  
&nbsp;  
&nbsp;  
&nbsp;  
&nbsp;  
# Docstring Generation Example 1

In [ ]:
def read_n_lines_at_a_time(f, n):
    while True:
        lines = [f.readline() for i in range(n)]
        lines = [line for line in lines if line != '']
        if len(lines) > 0:
            yield [line for line in lines if line != '']
        else:
            break

In [269]:
generate_docstring(example1)

read_n_lines_at_a_time(f, nn)

Reads the lines from the file f, which should be a open file object.

Parameters
----------
f : open file object
    The open file object from which to read the lines.
nn : int
    The number of lines to read.


&nbsp;  
&nbsp;  
&nbsp;  
&nbsp;  
&nbsp;  
&nbsp;  
&nbsp;  
&nbsp;  
&nbsp;  
&nbsp;  
&nbsp;  
&nbsp;  
&nbsp;  
&nbsp;  
&nbsp;  
&nbsp;  
&nbsp;  
&nbsp;  
&nbsp;  
&nbsp;  
# Docstring Generation Example 2

In [250]:
def grab_source_files_from_repo(repo, extension, quiet=True):
    all_raw_input_files = []
    try:
        if isinstance(extension, str):
            raw_input_files = Path(repo).glob('**/*.'+extension)
        else:
            extension = ['.'+ext for ext in extension]
            raw_input_files = Path(repo).glob('**/*')
            raw_input_files = [p for p in raw_input_files if grab_extension(p) in extension]
        res = [str(p) for p in raw_input_files if p.is_file()]
        return res
    except OSError:
        if not quiet:
            print("couldn't process "+str(repo))
        return []

In [270]:
generate_docstring(example2)

Get a list of source files from a repository.

:param repo: Path to the repository.
:type repo: str
:param extension: File extension.
:type extension: str
:param quiet: Suppress errors.
:type quiet: bool
:return: List of source files.
:rtype: list


&nbsp;  
&nbsp;  
&nbsp;  
&nbsp;  
&nbsp;  
&nbsp;  
&nbsp;  
&nbsp;  
&nbsp;  
&nbsp;  
&nbsp;  
&nbsp;  
&nbsp;  
&nbsp;  
&nbsp;  
&nbsp;  
&nbsp;  
&nbsp;  
&nbsp;  
&nbsp;  
# Docstring Generation Example 3

In [ ]:
def parse_manifest(files, dirs):
    f = open(MANIFEST_IN, 'r')
    for line in f:
        line = line.strip()
        if ((len(line) <= 0) or (line[0] == '#')):
            continue
        tokens = line.split()
        if (len(tokens) >= 0):
            if (tokens[0] == 'include'):
                files.extend(tokens[1:])
            elif ((tokens[0] == 'recursive-include') and (tokens[2] == '*')):
                dirs.append(tokens[1])
            else:
                err = (((('Unsupported type ' + tokens[0]) + ' in ') + MANIFEST_IN) + ' file.')
                raise Exception(err)
    f.close()

In [271]:
generate_docstring(example3)

Parses the contents of a MANIFEST_IN, a list of files to
include, and a mapping of directories to their contents.


&nbsp;  
&nbsp;  
&nbsp;  
&nbsp;  
&nbsp;  
&nbsp;  
&nbsp;  
&nbsp;  
&nbsp;  
&nbsp;  
&nbsp;  
&nbsp;  
&nbsp;  
&nbsp;  
&nbsp;  
&nbsp;  
&nbsp;  
&nbsp;  
&nbsp;  
&nbsp;  
# Docstring Generation Example 4

In [ ]:
def plot_grid(self, **kwargs):
    if ('ax' in kwargs):
        ax = kwargs.pop('ax')
    else:
        ax = self.ax
    if ('color' not in kwargs):
        kwargs['color'] = '0.5'
    lc = self.get_grid_line_collection(**kwargs)
    ax.add_collection(lc)
    return lc

In [272]:
generate_docstring(example4)

Add a grid to the current plot.

Parameters
----------
ax : matplotlib.axes.Axes, optional
    The axes to add the grid to.
color : string, optional
    The color of the grid lines.
ax : matplotlib.axes.Axes, optional
    The axes to add the grid to.

Returns
-------
ax : matplotlib.axes.Axes
